In [ ]:
import os
import time
import subprocess
import pickle
import ruamel.yaml
from ruamel.yaml.scalarstring import DoubleQuotedScalarString


In [ ]:
class OrcaWrapper:
    def __init__(self, orca_path='orca'):
        """
        Initialize the OrcaWrapper with the path to the ORCA executable.

        :param str orca_path: Path to the ORCA executable.
        """
        self.orca_path = orca_path

    def setup_calculation(self, method, command, params):
        """
        Set up the ORCA calculation by customizing a Kubernetes template.

        :param str method: Method used for the calculation.
        :param str command: Command to execute the ORCA calculation.
        :param dict params: Additional parameters for customizing the template.

        :return: Filename of the Kubernetes configuration file as a string.
        :return: Identifier for the ORCA calculation as a string.
        """
        orca_method_file = params.get('orca_method_file', '')
        timestamp = str(time.time()).replace(".", "")
        method = method.replace("_", "-")

        # Determine default image and name based on the method
        default_image, default_name = self._get_default_image_and_name(method, params)

        # Load Kubernetes template from file
        template_file = "orca-k8s-template.yaml" if method == "orca" else "gmx-k8s-template.yaml"
        with open(template_file) as ifile:
            doc = ruamel.yaml.round_trip_load(ifile, preserve_quotes=True)

            # Customize the template with parameters and settings
            self._customize_template(doc, method, command, params, default_image, default_name)

            # Generate an identifier for the ORCA calculation
            identificator = "{}-{}-rdtscp-{}".format(default_name, method, timestamp)

            # Write the customized template to a YAML file
            ofile_name = "{}-{}-rdtscp.yaml".format(default_name, method)
            with open(ofile_name, "w") as ofile:
                ruamel.yaml.round_trip_dump(doc, ofile, explicit_start=True)

            return ofile_name, identificator

    def run_calculation(self, method, orca_method, log, **kwargs):
        """
        Run the ORCA calculation by setting up Kubernetes job and executing it.

        :param str method: Method used for the calculation.
        :param str orca_method: ORCA method used for the computation.
        :param str log: Log file to store the output of the calculation.
        :kwargs: Additional parameters for customizing the calculation.
        """
        params = {
            "image": kwargs.get('image', None),
            "workdir": kwargs.get('workdir', None),
            "parallel": kwargs.get('parallel', None)
        }

        # Prepare ORCA command to execute the calculation
        orca_command = self._prepare_orca_command(orca_method, log, params)

        # Set up the calculation and run the Kubernetes job
        method_path = "{}/{}".format(params['workdir'], orca_method)
        kubernetes_config, label = self.setup_calculation(method, orca_command, params)
        print(self._run_job(kubernetes_config, label, params["parallel"]))

    def _get_default_image_and_name(self, method, params):
        """
        Determine the default Docker image and name based on the method.
        
        :param str method: Method used for the calculation.
        :param dict params: Additional parameters for customizing the calculation.
        :return: Default Docker image as a string.
        :return: Default name as a string.
        """
        if method == "orca":
            return 'ljocha/orca:5.0.1', 'orca'
        elif method == 'parmtsnecv':
            return Config.PARMTSNECV_IMAGE, 'parmtsnecv'
        else:
            return Config.GMX_IMAGE, 'gromacs'

    def _customize_template(self, doc, method, command, params, default_image, default_name):
        """
        Customize the Kubernetes template with parameters and settings.

        :param dict doc: Kubernetes template document.
        :param str method: Method used for the calculation.
        :param str command: Command to execute the calculation.
        :param dict params: Additional parameters for customizing the template.
        :param str default_image: Default Docker image.
        :param str default_name: Default name.
        """
        doc['spec']['template']['spec']['containers'][0]['image'] = default_image if not params["image"] else params["image"]
        doc['spec']['template']['spec']['containers'][0]['workingDir'] = "/tmp/"
        if params["workdir"]:
            doc['spec']['template']['spec']['containers'][0]['workingDir'] += params["workdir"]

    def _prepare_orca_command(self, orca_method, log, params):
        """
        Prepare the ORCA command for execution in the Kubernetes cluster.
        
        :param str orca_method: ORCA method used for the computation.
        :param str log: Log file to store the output of the computation.
        :param dict params: Dictionary containing additional parameters for the computation,
                            such as the working directory and parallel flag.
        :return: The prepared ORCA command as a string.
        """
        # Define the ORCA application and construct the command string
        application = "orca"
        orca_command = (
            f"mkdir -p /tmp/orca && "
            f"cp /share/{params['workdir']}/* /tmp/orca && "
            f"cd /tmp/orca && "
            f"/opt/orca/{application} {orca_method} > {log}; "
            f"cp /tmp/orca/* /share/{params['workdir']}"
        )
        return orca_command

    def _run_job(self, kubernetes_config, label, parallel):
        """
        Run the Kubernetes job for executing the ORCA computation.

        :param str kubernetes_config: The Kubernetes configuration file for the job.
        :param str label: The label for identifying the job.
        :param bool parallel: Flag indicating whether parallel execution is enabled.
        """
        # Check if parallel execution is enabled
        if parallel:
            # If parallel execution is enabled, modify the label to include the parallel identifier
            label += "-parallel"
    
        # Execute the Kubernetes job using kubectl
        try:
            # Example command: kubectl apply -f kubernetes_config.yaml --selector app=label
            subprocess.run(["kubectl", "apply", "-f", kubernetes_config, "--selector", f"app={label}"], check=True)
            print("Kubernetes job submitted successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Error submitting Kubernetes job: {e}")
            
    

In [ ]:
import os
import uuid
import tempfile

class OrcaRunnerK8s:
    orca_cmd = 'orca'

    def __init__(self, pvc=None, workdir=None, image='ljocha/orca:5.0.1', **kwargs):
        self.image = image

        # Heuristics to find PVC and working dir
        if pvc is None:
            vol, _, _, _, _, mnt = os.popen('df .').readlines()[1].split()
            pvcid = re.search('pvc-[0-9a-z-]+', vol).group(0)
            pvc = os.popen(f'kubectl get pvc | grep {pvcid} | cut -f1 -d" "').read().rstrip()

        if workdir is None:
            workdir = os.path.relpath(os.getcwd(), mnt)

        self.workdir = workdir
        self.pvc = pvc
        self.jobname = "orca-" + str(uuid.uuid4())

    def prehook(self, cores=None, mpi=1, omp=1, gpus=0, gputype='mig-1g.10gb', mem=4):
        # Logic to generate Kubernetes Job YAML configuration
        job_yaml = """
apiVersion: batch/v1
kind: Job
metadata:
  name: {jobname}
spec:
  backoffLimit: 0
  template:
    metadata:
      labels:
        job: {jobname}
    spec:
      restartPolicy: Never
      containers:
      - name: {jobname}
        image: {image}
        workingDir: /mnt/{workdir}
        command: 
        - sleep
        - 365d
        # Add security context, resources, volume mounts, etc.
      volumes:
      - name: vol-1
        persistentVolumeClaim:
          claimName: {pvc}
""".format(jobname=self.jobname, image=self.image, workdir=self.workdir, pvc=self.pvc)

        # Apply Kubernetes Job configuration
        with tempfile.NamedTemporaryFile('w+') as y:
            y.write(job_yaml)
            y.flush()
            os.system(f'kubectl apply -f {y.name}')
            os.system(f'kubectl wait --for=condition=ready pod -l job={self.jobname}')

    def posthook(self):
        # Cleanup Kubernetes Job
        os.system(f'kubectl delete job/{self.jobname}')

    def commandline(self, **kwargs):
        # Return the command line to execute commands in the Kubernetes pod
        return ['kubectl', 'exec', '-ti', f'job/{self.jobname}', '--', self.orca_cmd]